# Obtenção e tratamento de dados

## Obtenção e carregamento de dados  

In [ ]:
#importação de bibliotecas
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.manifold import TSNE
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.metrics import silhouette_score

In [ ]:
#Carregamento de dados limpos da pasta ../data/
df = pd.read_csv('../Data/dataset.csv')

## Informações sobre o dataset

In [ ]:
#Exibição dos dados
df.head()

In [ ]:
#Exibição de informações sobre os dados
df.info()

In [ ]:
#Exibição de estatísticas descritivas
df.describe()

## Ajuste de tipos

In [ ]:
#Matricula
df['dt_matricula'] = pd.to_datetime(df['dt_matricula'])
#Saida
df['dt_saida'] = pd.to_datetime(df['dt_saida'])
#Primeiro login
df['primeiro_login'] = pd.to_datetime(df['primeiro_login'])
#Ultimo login
df['ultimo_login'] = pd.to_datetime(df['ultimo_login'])

#Idade
df['idade'] = df['idade'].astype('int64')
#Total de logins
df['total_logins'] = df['total_logins'].astype('int64')
#Dias desde o primeiro login
df['dias_desde_primeiro_login'] = df['dias_desde_primeiro_login'].astype('int64')
# Quantidade Mestrado
df['qtd_professores_mestrado'] = df['qtd_professores_mestrado'].astype('int64')
# Quantidade Doutorado
df['qtd_professores_doutorado'] = df['qtd_professores_doutorado'].astype('int64')
# Quantidade Especialização
df['qtd_professores_especialista'] = df['qtd_professores_especialista'].astype('int64')
# Quantidade Graduado
df['qtd_professores_graduado'] = df['qtd_professores_graduado'].astype('int64')
# Quantidade de professores Masculino
df['qtd_professores_M'] = df['qtd_professores_M'].astype('int64')
# Quantidade de professores Feminino
df['qtd_professores_F'] = df['qtd_professores_F'].astype('int64')
# Menor idade de um professor
df['idade_min_professores'] = df['idade_min_professores'].astype('int64')
# Maior idade de um professor
df['idade_max_professores'] = df['idade_max_professores'].astype('int64')

#Data de registro para string
df['dt_registro'] = df['dt_registro'].astype(object)
#Data de inauguração do curso
df['dt_inauguracao_curso'] = df['dt_inauguracao_curso'].astype(object)

#Exibição de informações sobre os dados
df.info()
#Exibição de estatísticas descritivas
df.head()


## Tratamento de valores ausentes

In [ ]:
#preenche com o texto "Abandono" os valores nulos da coluna 'categoria_motivo_cancelamento'
df['categoria_motivo_cancelamento'] = df['categoria_motivo_cancelamento'].fillna('Abandono').where(df['evasao'] == 1,'NA')

#Exibe os valores unicos da coluna 'categoria_motivo_cancelamento'
df['categoria_motivo_cancelamento'].unique()

Exibição dos dados

In [ ]:
#Exibe as informações sobre os dados
df.info()
#Exibe as estatísticas descritivas
df.head()

# Aprendizado de Maquina (Clusterização - KMeans + T-SNE)

## Amostra dos dados

In [ ]:
df.head()

## Remoção de variaveis correlacionadas

In [ ]:
# Exclua as colunas que não serão usadas
df = df.drop(['cd_aluno','nome', 'curso','cidade','titulacao', 'estado', 'semestre_atual','dias_matriculado_semestre','total_logins','qtd_aprovacoes','valor_mensalidade_bruto', 'percentual_desconto','dias_desde_primeiro_login', 'dt_registro', 'dt_inauguracao_curso','dt_matricula', 'dt_saida', 'primeiro_login', 'ultimo_login', 'idade_min_professores', 'idade_max_professores'], axis=1)

## Separação de variaveis categoricas e numericas

In [ ]:
# Defina as colunas numéricas e categóricas
num_cols = (df.select_dtypes(include=['int64', 'float64']).columns).where(df.select_dtypes(include=['int64', 'float64']).columns != 'evasao').dropna().tolist()
cat_cols = ['genero', 'ds_ensino_medio', 'ds_ingresso', 'cd_curso', 'situacao', 'categoria_motivo_cancelamento']

## Normalização e codificação das variaveis

### Normalização das variáveis numericas

In [ ]:
# Cria uma cópia do DataFrame principal para normalizar as colunas numéricas
df_norm = df.copy()

# Cria uma instância do StandardScaler
scaler = StandardScaler()
# Normaliza as colunas numéricas e substitui no DataFrame normalizado
df_norm[num_cols] = scaler.fit_transform(df_norm[num_cols])

#Adiciona a coluna 'evasao' ao DataFrame normalizado na primeira posição
df_norm = pd.concat([df_norm['evasao'], df_norm.drop('evasao', axis=1)], axis=1)

df_norm.head()

### Codificação das variáveis categoricas

In [ ]:
# Cria uma instância do OneHotEncoder com a opção de remover a primeira coluna
encoder = OneHotEncoder(drop='first')

# Codifica as colunas categóricas
df_encoded = encoder.fit_transform(df_norm[cat_cols])

# Criando os nomes das colunas
column_names = encoder.get_feature_names_out(cat_cols)

# Criando um dataframe com os dados codificados
df_encoded = pd.DataFrame(df_encoded.toarray(), columns=column_names)

# Concatenando os dataframes
df_norm = pd.concat([df_norm, df_encoded], axis=1)

# Removendo as colunas categóricas originais
df_norm = df_norm.drop(cat_cols, axis=1)

# Exibe as informações sobre os dados
df_norm.info()

# Exibe uma amostra dos dados
df_norm.head(10)

## Remoção de variavel alvo

In [ ]:
# Removendo temporariamente a coluna 'evasao' para não influenciar o modelo
df_sem_alvo = df_norm.drop('evasao', axis=1)

#Cria uma copia do dataframe principal para ser usado no modelo
df_modelo = df_norm.copy()

df_modelo.head()

## Redução de dimensionalidade

In [ ]:
# Ajuste do t-SNE
tsne = TSNE(n_components=2, random_state=42, perplexity=40, learning_rate=150,n_iter=2000, metric='cosine')
df_tsne = tsne.fit_transform(df_sem_alvo)

# Convertendo o resultado do t-SNE em um DataFrame
df_tsne = pd.DataFrame(df_tsne, columns=['x', 'y'])

# Concatenando os dataframes
df_norm = pd.concat([df_norm, df_tsne], axis=1)

# Atribui os valores do t-SNE ao dataframe usado para o modelo
df_modelo = df_tsne.copy()

df_modelo.head()

## Definição do número de clusters (Elbow Method)

In [ ]:
# Método do cotovelo#
wcss = []
# Testando de 1 a 10 clusters
for i in range(1, 11):
    kmeans = KMeans(n_clusters=i, init='k-means++', max_iter=300, n_init=10, random_state=0)
    kmeans.fit(df_norm)
    wcss.append(kmeans.inertia_)

# Exibindo o método do cotovelo
plt.figure(figsize=(10, 6))
plt.plot(range(1, 11), wcss, marker='o')
plt.title('Método do Cotovelo')
plt.xlabel('Número de clusters')
plt.xticks(range(1, 11))
plt.ylabel('WCSS')
plt.show()

## Ajuste do modelo

In [ ]:
# Ajuste do modelo de clusterização nos dados transformados
kmeans = KMeans(n_clusters=5, n_init=10, random_state=42)
kmeans.fit(df_modelo)

# Adicionando os labels dos clusters ao dataframe normalizado
df_norm['cluster'] = kmeans.labels_

# Adicionando os labels dos clusters ao dataframe principal
df['cluster'] = kmeans.labels_

df.info()

## Avaliação do modelo

### Silhouette Score

In [ ]:
# Calculando a pontuação da silhueta
score = silhouette_score(df_modelo, kmeans.labels_)

print('Silhouette Score: ', score)

## Visualização dos clusters

### Visualização dos clusters em 2D

In [ ]:
# Criando uma figura e um eixo
fig, ax = plt.subplots( figsize=(10, 7))

# Criando uma lista de cores para cada cluster
colors=[ '#FFC75F', '#FF9671', '#FF6F91', '#D65DB1', '#845EC2']

# Plotando cada cluster
for i in range(5):
    # Selecionando apenas os pontos de dados neste cluster
    cluster = df_norm[df_norm['cluster'] == i]
    ax.scatter(cluster['x'], cluster['y'], c=colors[i], label=f'Cluster {i}', alpha=0.5)

# Adicionando uma legenda
ax.legend()
plt.title('Visualização dos Clusters de estudantes com t-SNE')
plt.show()

### Médias dos clusters

In [ ]:
# Adiciona a coluna 'evasao' a lista de colunas numericas
num_cols.insert(0, 'evasao')

#Atribui a variavel 'cluster_means' a média de cada coluna numerica (num_cols) + a coluna evasao por cluster do DataFrame 'df'
cluster_means = df.groupby('cluster')[num_cols].mean()

# Exibindo a média de cada coluna por cluster
cluster_means.head()

## Exportação dos resultados

In [ ]:
df.to_csv('../data/dataset_clustered.csv', index=False)

df.info()